In [1]:
import json
task = 'Common_neighbors'
filename = task+'_result_1shot.json'
with open(filename, 'r', encoding='utf-8') as f:
    datas=json.load(f)

In [2]:
filename1 = task+'_test.json'
with open(filename1, 'r', encoding='utf-8') as f:
    gt=json.load(f)

In [3]:
print(len(datas))
print(len(gt))
print(gt[0])

500
500
{'index': 0, 'input_prompt': 'Your task is to find the common neighbors of two nodes in an undirected academic network. In this network, nodes represent authors and edges represent research collaborations.\n\n**Problem to Solve**\n\n- Authors in the network: Manfred Schmidt-Schauss, David Sabel, Markus Lohrey, Sebastian Maneth, Conrad Rau, Manfred Schmidt-Schauß, Jordi Levy\n- Research collaborations between these authors: Manfred Schmidt-Schauss and Jordi Levy, Manfred Schmidt-Schauss and David Sabel, Manfred Schmidt-Schauss and Manfred Schmidt-Schauß, Manfred Schmidt-Schauss and Markus Lohrey, Manfred Schmidt-Schauss and Sebastian Maneth, Manfred Schmidt-Schauss and Conrad Rau, David Sabel and Conrad Rau, David Sabel and Manfred Schmidt-Schauß, Markus Lohrey and Manfred Schmidt-Schauß, Markus Lohrey and Jordi Levy, Markus Lohrey and Sebastian Maneth, Sebastian Maneth and Manfred Schmidt-Schauß, Sebastian Maneth and Jordi Levy, Conrad Rau and Manfred Schmidt-Schauß, Manfred Sc

In [4]:
import re
import io
import sys


In [5]:
print(gt[16]['input_prompt'])
print(datas[16]['code'])
print(gt[16]['answer'])

# code = re.sub(r'^```python\n|```$', '', datas[5]['code'], flags=re.MULTILINE)
# code = code.split('[Code]:')[1]
# print(code)

Your task is to find the common neighbors of two nodes in an undirected academic network. In this network, nodes represent authors and edges represent research collaborations.

**Problem to Solve**

- Authors in the network: Jolie M. Martin, Varun Dutt, Cleotilde Gonzalez, Christian Lebiere
- Research collaborations between these authors: Jolie M. Martin and Cleotilde Gonzalez, Jolie M. Martin and Christian Lebiere, Varun Dutt and Cleotilde Gonzalez, Varun Dutt and Christian Lebiere, Cleotilde Gonzalez and Christian Lebiere.
Please identify the common neighbors of Jolie M. Martin and Varun Dutt in this network.
You only need to output the number of the common neighbors. 
1. Identify the nodes in the graph: Jolie M. Martin, Varun Dutt, Cleotilde Gonzalez, Christian Lebiere.
2. Identify the edges in the graph: Jolie M. Martin-Cleotilde Gonzalez, Jolie M. Martin-Christian Lebiere, Varun Dutt-Cleotilde Gonzalez, Varun Dutt-Christian Lebiere, Cleotilde Gonzalez-Christian Lebiere.
3. Convert

In [6]:
    import networkx as nx

    edge_list = [('Jolie M. Martin', 'Cleotilde Gonzalez'), ('Jolie M. Martin', 'Christian Lebiere'), ('Varun Dutt', 'Cleotilde Gonzalez'), ('Varun Dutt', 'Christian Lebiere'), ('Cleotilde Gonzalez', 'Christian Lebiere')]
    G = nx.Graph(edge_list)
    print(len(list(nx.common_neighbors(G, 'Jolie M. Martin', 'Varun Dutt'))))

2


In [7]:
error_cnt = 0
error_cnt_code = 0
correct_cnt = 0
for i, data in enumerate(datas):
    text = data['code']
    #code = text
    code = re.sub(r'^```python\n|```$', '', text, flags=re.MULTILINE)
    
    if '[Code]:' in code:
        code = code.split('[Code]:')[1]
    elif '**Code:**' in code:
        code = code.split('**Code:**')[1]
    elif '### Code' in code:
        code = code.split('### Code')[1]
    elif '**Code**' in code:
        code = code.split('**Code**')[1]   
    elif '**Code**:' in code:
        code = code.split('**Code**:')[1] 
    elif '### Solution' in code:
        code = code.split('### Solution')[1] 
    else:
        error_cnt = error_cnt + 1
        print(code)
        continue

    ans = gt[i]['answer']

    # 创建一个 StringIO 对象来捕获输出
    output = io.StringIO()

    # 重定向 sys.stdout
    old_stdout = sys.stdout
    sys.stdout = output

    try:
        exec(code, globals(), globals())
    except:
        sys.stdout = old_stdout
        #print(code)
        error_cnt_code = error_cnt_code + 1
        continue
    finally:
        sys.stdout = old_stdout
    

    # 获取输出内容
    output_content = output.getvalue()
    output_content = str(output_content).strip()
    # print("Captured Output:")
    #print(output_content)
    
    
    # print(f'llm={output_content} gt={ans}')
    try:
        if int(output_content) == int(ans):
            correct_cnt = correct_cnt + 1
        else:
            pass
            #print(code)
            #print(output_content,ans)
    except:
        print("**************")
        print(i)
        #print(output_content,ans)

In [8]:
print(error_cnt)
print(correct_cnt)
print(error_cnt_code)

0
438
60


In [9]:
print(correct_cnt/len(datas))
#max tokens是

0.876


In [9]:
import re
import networkx as nx

# Graph description
graph = """Node 6 is connected to nodes 5, 2, 0, 3, 1.
Node 5 is connected to nodes 2, 3.
Node 2 is connected to nodes 3, 1, 5.
Node 0 is connected to nodes 4, 3, 5.
Node 3 is connected to nodes 1, 6, 5, 2, 0, 4.
Node 1 is connected to node 5.
Node 4 is connected to nodes 7, 3, 5, 2.
Node 7 is connected to nodes 5, 0."""

# Regular expression to extract connections
pattern = r'Node (\d+) is connected to nodes? ([\d, ]+)\.'
matches = re.findall(pattern, graph)

# Create a directed graph
G = nx.DiGraph()

# Add edges to the graph
for node, neighbors in matches:
    node = int(node)
    neighbors = map(int, neighbors.split(', '))
    for neighbor in neighbors:
        G.add_edge(node, neighbor)
print(G.edges())
# Calculate clustering coefficient for node 6
node1 = 6
print(nx.clustering(G, node1))

[(6, 5), (6, 2), (6, 0), (6, 3), (6, 1), (5, 2), (5, 3), (2, 3), (2, 1), (2, 5), (0, 4), (0, 3), (0, 5), (3, 1), (3, 6), (3, 5), (3, 2), (3, 0), (3, 4), (1, 5), (4, 7), (4, 3), (4, 5), (4, 2), (7, 5), (7, 0)]
0.6785714285714286


In [10]:
import re
import networkx as nx

# Graph description
graph = """Node 4 is connected to nodes 2, 5.
Node 2 is connected to nodes 8, 5, 1.
Node 5 is connected to nodes 3, 6.
Node 3 is connected to nodes 0, 7, 6, 5, 4.
Node 0 is connected to nodes 8, 4.
Node 7 is connected to nodes 1, 2.
Node 6 is connected to nodes 8, 5, 1.
Node 1 is connected to node 2.
Node 8 is connected to node 6."""

# Regular expression to extract connections
pattern = r'Node (\d+) is connected to nodes? ([\d, ]+)\.'
matches = re.findall(pattern, graph)

# Create a directed graph
G = nx.DiGraph()

# Add edges to the graph
for node, neighbors in matches:
    node = int(node)
    neighbors = map(int, neighbors.split(', '))
    for neighbor in neighbors:
        G.add_edge(node, neighbor)

# Calculate out-degree of node 3
out_degree = nx.out_degree(G, node3)
print(out_degree)

AttributeError: module 'networkx' has no attribute 'out_degree'

In [10]:
import networkx as nx

edge_list = [('Yihua Shi', 'Yilong Yin'), ('Yihua Shi', 'Gongping Yang'), ('Yihua Shi', 'Rongyang Xiao'), ('Yihua Shi', 'Lu Yang'), ('Yihua Shi', 'Ju Cheng Yang'), ('Yihua Shi', 'Shan Juan Xie'), ('Yihua Shi', 'Dong Sun Park'), ('Yihua Shi', 'Xiaoming Xi'), ('Yihua Shi', 'Anil K. Jain'), ('Yihua Shi', 'Naoto Miura'), ('Yihua Shi', 'Takafumi Miyatake'), ('Anil K. Jain', 'Yilong Yin'), ('Anil K. Jain', 'Gongping Yang'), ('Anil K. Jain', 'Lu Yang'), ('Anil K. Jain', 'Ju Cheng Yang'), ('Anil K. Jain', 'Shan Juan Xie'), ('Anil K. Jain', 'Dong Sun Park'), ('Anil K. Jain', 'Xiaoming Xi'), ('Anil K. Jain', 'Ramachandra Raghavendra'), ('Rongyang Xiao', 'Yilong Yin'), ('Rongyang Xiao', 'Gongping Yang'), ('Rongyang Xiao', 'Xiaoming Xi'), ('Rongyang Xiao', 'Naoto Miura'), ('Rongyang Xiao', 'Takafumi Miyatake'), ('Rongyang Xiao', 'Lu Yang'), ('Lu Yang', 'Yilong Yin'), ('Lu Yang', 'Gongping Yang'), ('Lu Yang', 'Ju Cheng Yang'), ('Lu Yang', 'Shan Juan Xie'), ('Lu Yang', 'Dong Sun Park'), ('Lu Yang', 'Xiaoming Xi'), ('Lu Yang', 'Ramachandra Raghavendra'), ('Lu Yang', 'Naoto Miura'), ('Lu Yang', 'Hee Chan Kim'), ('Lu Yang', 'Joon Hwan Choi'), ('Lu Yang', 'Takafumi Miyatake'), ('Lu Yang', 'Won-Seok Song'), ('Lu Yang', 'Seung-Rae Lee'), ('Lu Yang', 'Xianjing Meng'), ('Naoto Miura', 'Yilong Yin'), ('Naoto Miura', 'Gongping Yang'), ('Naoto Miura', 'Ju Cheng Yang'), ('Naoto Miura', 'Shan Juan Xie'), ('Naoto Miura', 'Dong Sun Park'), ('Naoto Miura', 'Xiaoming Xi'), ('Naoto Miura', 'Ramachandra Raghavendra'), ('Ju Cheng Yang', 'Yilong Yin'), ('Ju Cheng Yang', 'Gongping Yang'), ('Ju Cheng Yang', 'Shan Juan Xie'), ('Ju Cheng Yang', 'Dong Sun Park'), ('Ju Cheng Yang', 'Takafumi Miyatake'), ('Xianjing Meng', 'Yilong Yin'), ('Xianjing Meng', 'Gongping Yang'), ('Xianjing Meng', 'Shan Juan Xie'), ('Xianjing Meng', 'Dong Sun Park'), ('Yilong Yin', 'Gongping Yang'), ('Yilong Yin', 'Shan Juan Xie'), ('Yilong Yin', 'Dong Sun Park'), ('Yilong Yin', 'Xiaoming Xi'), ('Yilong Yin', 'Ramachandra Raghavendra'), ('Yilong Yin', 'Hee Chan Kim'), ('Yilong Yin', 'Joon Hwan Choi'), ('Yilong Yin', 'Takafumi Miyatake'), ('Yilong Yin', 'Won-Seok Song'), ('Yilong Yin', 'Seung-Rae Lee'), ('Gongping Yang', 'Shan Juan Xie'), ('Gongping Yang', 'Dong Sun Park'), ('Gongping Yang', 'Xiaoming Xi'), ('Gongping Yang', 'Ramachandra Raghavendra'), ('Gongping Yang', 'Hee Chan Kim'), ('Gongping Yang', 'Joon Hwan Choi'), ('Gongping Yang', 'Takafumi Miyatake'), ('Gongping Yang', 'Won-Seok Song'), ('Gongping Yang', 'Seung-Rae Lee'), ('Hee Chan Kim', 'Shan Juan Xie'), ('Hee Chan Kim', 'Dong Sun Park'), ('Hee Chan Kim', 'Xiaoming Xi'), ('Xiaoming Xi', 'Joon Hwan Choi'), ('Xiaoming Xi', 'Takafumi Miyatake'), ('Xiaoming Xi', 'Won-Seok Song'), ('Xiaoming Xi', 'Seung-Rae Lee'), ('Won-Seok Song', 'Shan Juan Xie'), ('Won-Seok Song', 'Dong Sun Park'), ('Seung-Rae Lee', 'Shan Juan Xie'), ('Seung-Rae Lee', 'Dong Sun Park'), ('Ramachandra Raghavendra', 'Takafumi Miyatake'), ('Joon Hwan Choi', 'Shan Juan Xie'), ('Joon Hwan Choi', 'Dong Sun Park'), ('Takafumi Miyatake', 'Shan Juan Xie'), ('Takafumi Miyatake', 'Dong Sun Park'), ('Shan Juan Xie', 'Dong Sun Park')]

G = nx.Graph()
G.add_edges_from(edge_list)

common_neighbors = nx.common_neighbors(G, 'Yihua Shi', 'Anil K. Jain')
print(common_neighbors)

{'Xiaoming Xi', 'Shan Juan Xie', 'Gongping Yang', 'Yilong Yin', 'Lu Yang', 'Ju Cheng Yang', 'Dong Sun Park'}


In [12]:
import networkx as nx

edge_list = [('Yihua Shi', 'Yilong Yin'), ('Yihua Shi', 'Gongping Yang'), ('Yihua Shi', 'Rongyang Xiao'), ('Yihua Shi', 'Lu Yang'), ('Yihua Shi', 'Ju Cheng Yang'), ('Yihua Shi', 'Shan Juan Xie'), ('Yihua Shi', 'Dong Sun Park'), ('Yihua Shi', 'Xiaoming Xi'), ('Yihua Shi', 'Anil K. Jain'), ('Yihua Shi', 'Naoto Miura'), ('Yihua Shi', 'Takafumi Miyatake'), ('Anil K. Jain', 'Yilong Yin'), ('Anil K. Jain', 'Gongping Yang'), ('Anil K. Jain', 'Lu Yang'), ('Anil K. Jain', 'Ju Cheng Yang'), ('Anil K. Jain', 'Shan Juan Xie'), ('Anil K. Jain', 'Dong Sun Park'), ('Anil K. Jain', 'Xiaoming Xi'), ('Anil K. Jain', 'Ramachandra Raghavendra'), ('Rongyang Xiao', 'Yilong Yin'), ('Rongyang Xiao', 'Gongping Yang'), ('Rongyang Xiao', 'Xiaoming Xi'), ('Rongyang Xiao', 'Naoto Miura'), ('Rongyang Xiao', 'Takafumi Miyatake'), ('Rongyang Xiao', 'Lu Yang'), ('Lu Yang', 'Yilong Yin'), ('Lu Yang', 'Gongping Yang'), ('Lu Yang', 'Ju Cheng Yang'), ('Lu Yang', 'Shan Juan Xie'), ('Lu Yang', 'Dong Sun Park'), ('Lu Yang', 'Xiaoming Xi'), ('Lu Yang', 'Ramachandra Raghavendra'), ('Lu Yang', 'Naoto Miura'), ('Lu Yang', 'Hee Chan Kim'), ('Lu Yang', 'Joon Hwan Choi'), ('Lu Yang', 'Takafumi Miyatake'), ('Lu Yang', 'Won-Seok Song'), ('Lu Yang', 'Seung-Rae Lee'), ('Lu Yang', 'Xianjing Meng'), ('Naoto Miura', 'Yilong Yin'), ('Naoto Miura', 'Gongping Yang'), ('Naoto Miura', 'Ju Cheng Yang'), ('Naoto Miura', 'Shan Juan Xie'), ('Naoto Miura', 'Dong Sun Park'), ('Naoto Miura', 'Xiaoming Xi'), ('Naoto Miura', 'Ramachandra Raghavendra'), ('Ju Cheng Yang', 'Yilong Yin'), ('Ju Cheng Yang', 'Gongping Yang'), ('Ju Cheng Yang', 'Shan Juan Xie'), ('Ju Cheng Yang', 'Dong Sun Park'), ('Ju Cheng Yang', 'Takafumi Miyatake'), ('Xianjing Meng', 'Yilong Yin'), ('Xianjing Meng', 'Gongping Yang'), ('Xianjing Meng', 'Shan Juan Xie'), ('Xianjing Meng', 'Dong Sun Park'), ('Yilong Yin', 'Gongping Yang'), ('Yilong Yin', 'Shan Juan Xie'), ('Yilong Yin', 'Dong Sun Park'), ('Yilong Yin', 'Xiaoming Xi'), ('Yilong Yin', 'Ramachandra Raghavendra'), ('Yilong Yin', 'Hee Chan Kim'), ('Yilong Yin', 'Joon Hwan Choi'), ('Yilong Yin', 'Takafumi Miyatake'), ('Yilong Yin', 'Won-Seok Song'), ('Yilong Yin', 'Seung-Rae Lee'), ('Gongping Yang', 'Shan Juan Xie'), ('Gongping Yang', 'Dong Sun Park'), ('Gongping Yang', 'Xiaoming Xi'), ('Gongping Yang', 'Ramachandra Raghavendra'), ('Gongping Yang', 'Hee Chan Kim'), ('Gongping Yang', 'Joon Hwan Choi'), ('Gongping Yang', 'Takafumi Miyatake'), ('Gongping Yang', 'Won-Seok Song'), ('Gongping Yang', 'Seung-Rae Lee'), ('Hee Chan Kim', 'Shan Juan Xie'), ('Hee Chan Kim', 'Dong Sun Park'), ('Hee Chan Kim', 'Xiaoming Xi'), ('Xiaoming Xi', 'Joon Hwan Choi'), ('Xiaoming Xi', 'Takafumi Miyatake'), ('Xiaoming Xi', 'Won-Seok Song'), ('Xiaoming Xi', 'Seung-Rae Lee'), ('Won-Seok Song', 'Shan Juan Xie'), ('Won-Seok Song', 'Dong Sun Park'), ('Seung-Rae Lee', 'Shan Juan Xie'), ('Seung-Rae Lee', 'Dong Sun Park'), ('Ramachandra Raghavendra', 'Takafumi Miyatake'), ('Joon Hwan Choi', 'Shan Juan Xie'), ('Joon Hwan Choi', 'Dong Sun Park'), ('Takafumi Miyatake', 'Shan Juan Xie'), ('Takafumi Miyatake', 'Dong Sun Park'), ('Shan Juan Xie', 'Dong Sun Park')]

G = nx.Graph(edge_list)
print(len(list(nx.common_neighbors(G, 'Yihua Shi', 'Anil K. Jain'))))

7


In [38]:

import networkx as nx

edge_list = [('S. K. Samanta', 'Zeshui Xu'), ('S. K. Samanta', 'A. Ghareeb'), ('Zeshui Xu', 'Tapas Mondal'), ('Wenzhong Shi', 'A. Ghareeb'), ('A. Ghareeb', 'Tapas Mondal')]

G = nx.Graph(edge_list)
print(len(list(nx.common_neighbors(G, 'S. K. Samanta', 'Wenzhong Shi'))))

SyntaxError: incomplete input (1739189855.py, line 6)